In [ ]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /user

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 911kB 6.9MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archiv

#라이브러리 import

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException


chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

#news_scraping(): 뉴스 스크래핑 함수

In [ ]:
'''
def news_scraping(news_url,wd):
  title=wd.find_element_by_id('articleTitle').text
  datetime=wd.find_element_by_class_name('t11').text
  print('뉴스:',[title,datetime,news_url])

  return [title,datetime,news_url]
  '''

#comments_scraping():뉴스 댓글 스크래핑

In [ ]:
def comments_scraping(news_url,wd):
  title=wd.find_element_by_id('articleTitle').text
  datetime=wd.find_element_by_class_name('t11').text
  print('뉴스:',[title,datetime,news_url])

  try:
    btn_view=wd.find_element_by_class_name('u_cbox_btn_view_comment').click()
    #print('[댓글 더보기]',end='')
    time.sleep(1)
    
    while True:
      wd.find_element_by_class_name('u_cbox_page_more').click()
      #print('[더보기]',end='')
      time.sleep(1)

  except:
    pass


  #btn_reply_list=wd.find_elements_by_class_name('u_cbox_btn_reply')
  #for btn_reply in btn_reply_list:
  #  btn_reply.send_keys('\n')
  #  print('[답글]',end='')
  #  time.sleep(1)

  #print('[댓글 스크래핑]')
  comments_idx=0
  comments_df=pd.DataFrame(columns=['Title','date','Contents','Datetime','URL'])

  comments=wd.find_elements_by_class_name('u_cbox_comment_box')
  for comment in comments:
    try:
      date=comment.find_element_by_class_name('u_cbox_date').text
      contents=comment.find_element_by_class_name('u_cbox_contents').text
      #print(f' 댓글 #{comments_idx+1}:',[contents,date,news_url])
      comments_df.loc[comments_idx]=[title, datetime, contents,date,news_url]
      comments_idx+=1
    except NoSuchElementException:
      #print('[삭제되거나 부적절한 댓글]')
      continue

  return comments_df

#scraping():스크래핑 함수

In [ ]:
#191120~191130:검색결과 약 14,600,000개 (0.38초) 15->8
#191201~191231:검색결과 약 37,300,000개 (0.26초) 37->19
#200101~200131:검색결과 약 52,000,000개 (0.31초)  52->26
#200201~200229:검색결과 약 37,700,000개 (0.31초)  38->19
#200301~200331:검색결과 약 136,000,000개 (0.34초) 136->68
#200401~200430:검색결과 약 140,000,000개 (0.32초) 140->70
#200501~200531:검색결과 약 73,300,000개 (0.30초) 73->37
#200601~200630:검색결과 약 107,000,000개 (0.35초) 107->54
#200701~200731:검색결과 약 99,500,000개 (0.32초) 100->50
#200801~200831:검색결과 약 129,000,000개 (0.34초) 129->65
#200901~200930:검색결과 약 62,800,000개 (0.34초) 63->32
#201001~201031:검색결과 약 165,000,000개 (0.38초) 165->83
#201101~201130:검색결과 약 81,800,000개 (0.47초) 82->42
#201201~201217:검색결과 약 769,000개 (0.40초)   ->4


def scraping():
    wd=webdriver.Chrome('chromedriver',options=chrome_options)
    wd.implicitly_wait(3)
  
    wd.execute_script('window.open("about:blank","_blank");')
    tabs=wd.window_handles
    wd.switch_to.window(tabs[0])
    query='코로나'  #input('검색어 입력: ')
    sort = '0'   #관련도순=0  최신순=1  오래된순=2
    s_date = '2020.11.01'  #input("시작날짜 입력(20xx.xx.xx):")  
    e_date = '2020.11.30'    #input("끝날짜 입력(20xx.xx.xx):")
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    search_url="https://search.naver.com/search.naver?where=news&query=" + query + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=0" 
    wd.get(search_url)


    comments_df=pd.DataFrame()
    
    page=21
    while page!=0:
        article_list=wd.find_elements_by_css_selector('ul.list_news > li.bx')
        len(article_list)
        target_naver_news_href=[]

        for this_article in article_list:
            naver_news_check=this_article.find_elements_by_css_selector('div.news_wrap.api_ani_send > div.news_area > div > div > a')
            len(naver_news_check)
            for this_cand in naver_news_check:
                if this_cand.text=="네이버뉴스":
                    target_naver_news_href.append(this_cand.get_attribute('href'))
        #print(target_naver_news_href)
        
        while target_naver_news_href!=[]:
            try:
                wd.switch_to.window(tabs[1])
                get_url=target_naver_news_href.pop(0)
                wd.get(get_url)

                df=comments_scraping(get_url,wd)
                #comments_df.loc[comments_idx]=[query]
                comments_df=pd.concat([df,comments_df])
                

            except:
                continue
            
        
  
        try:
            wd.switch_to.window(tabs[0])
            nextpage=wd.find_element_by_class_name('btn_next').get_attribute('href')
            wd.get(nextpage)
            print('[다음페이지]')
            time.sleep(1)
            page-=1

        except:
            break      

    return comments_df

In [ ]:
comments_df=scraping()

뉴스: ['부산시 "수능일까지 코로나 대응 3단계 수준으로 대폭 강화"(종합)', '2020.11.30. 오전 11:58', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0012048463']
뉴스: ['정은경 "1~2주 후 코로나 확진자 하루 700~1000명 가능"(상보)', '2020.11.30. 오후 3:50', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=277&aid=0004801303']
뉴스: ['[종합] "우리 코로나 취급하더니"…野, \'조기축구\' 최재성에 \'부글\'', '2020.11.30. 오후 4:16', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=277&aid=0004801335']
뉴스: ['종근당 "코로나 치료제 내년 1월 국내 허가 신청"…러시아 임상서 효과', '2020.11.30. 오전 10:32', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=215&aid=0000919039']
뉴스: ['수원지검, 검사 1명 코로나19 확진…현직 검사 첫 사례', '2020.11.30. 오후 8:52', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=018&aid=0004797502']
뉴스: ['“코로나 블루 같아요”…국민 100명 중 5명 우울증, 10년새 증가', '2020.11.30. 오전 11:31', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=081&aid=0003143745']
뉴스: ['코로나 3차유행 본격화…"내년에도 채용 축소·재택근무"', '202

In [ ]:
data = pd.DataFrame(comments_df)  
data.to_csv('뉴스크롤링결과코로나2011.csv', encoding='utf8')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')